In [1]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt

## Load Data

In [8]:
# dota format
""" collision event sample
{
    "actutorId":92,
    "timestamp":43.79999923706055,
    "duration":1.0400009155273438,
    "other":"shape1",
    "detail":"Cube.029"
},
"""

' collision event sample\n{\n    "actutorId":92,\n    "timestamp":43.79999923706055,\n    "duration":1.0400009155273438,\n    "other":"shape1",\n    "detail":"Cube.029"\n},\n'

In [2]:
num_user = 9
folder_paths = ["OhShape/user3/", "OhShape/user4/", "OhShape/user5/", "OhShape/user6/", "OhShape/user7/", "OhShape/user8/", "OhShape/user9/", "OhShape/user10/", "OhShape/user11/"]
num_mode = 4 # PRACTICE, CLEAN, VISUAL, HAPTIC
mode_list = ["PRACTICE", "CLEAN", "VISUAL", "HAPTIC"]
num_actuator = 116
actuator_ids = [ 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
                31, 32, 33, 34, 35, 36, 37, 38, 39, 40,
                61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80, 
                91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 
                121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 
                151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 
                181, 182, 183, 184, 185, 186, 187, 188,
                211, 212, 213, 214, 215, 216, 217, 218]

actuator_collision_count_array = np.zeros((num_user, num_actuator), dtype=np.int32)
collision_count_array_prac = np.zeros((num_user, num_mode), dtype=np.int32)
collision_count_array_test = np.zeros((num_user, num_mode), dtype=np.int32)
collision_duration_array_test = np.zeros((num_user, num_mode), dtype=np.float32)


In [3]:
def remove_duplicate_entry(json_objects):
    # Convert each JSON object to a string and add it to a set to remove duplicates
    unique_json_objects_str = {json.dumps(d, sort_keys=True) for d in json_objects}

    # Convert each unique JSON string back to a JSON object
    unique_json_objects = [json.loads(s) for s in unique_json_objects_str]
    
    return unique_json_objects

In [4]:
for user_index in range(num_user):
    folder_path = folder_paths[user_index]
    print(f"loading {folder_path}")
    
    # initialize
    actuator_array = {"PRACTICE":[], "CLEAN":[], "VISUAL":[], "HAPTIC":[]}
    duration_array = {"PRACTICE":[], "CLEAN":[], "VISUAL":[], "HAPTIC":[]}
    wall_array = {"PRACTICE":[], "CLEAN":[], "VISUAL":[], "HAPTIC":[]}
    shape_array = {"PRACTICE":[], "CLEAN":[], "VISUAL":[], "HAPTIC":[]}
    
    json_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) and f.endswith('.json')]
    
    # load data
    for json_file in json_files:
        with open(os.path.join(folder_path, json_file), 'r') as file:
            data = json.load(file)
        mode = data["mode"]
        shape_indexs = data["shapeIndexs"]
        collision_count = data["numberOfCollusions"]
        collision_data = remove_duplicate_entry(data['collusionData']) # remove duplicate entry
        print(f"load {json_file}, mode = {mode}, collision count = {collision_count}, after removing duplicate = {len(collision_data)}")
        for i in range(len(collision_data)):
            actuator_array[mode].append(collision_data[i]['actutorId'])
            duration_array[mode].append(collision_data[i]['duration'])
            wall_index = int(collision_data[i]['other'].replace('shape', ''))
            wall_array[mode].append(wall_index)
            shape_array[mode].append(shape_indexs[wall_index])
    
    # compare the length of practice and test data, showing the learning effect
    # only keep the test wall data
    for mode_index in range(len(mode_list)):
        mode = mode_list[mode_index]
        prac_wall_indices = np.where(np.array(wall_array[mode]) % 2 == 0)
        test_wall_indices = np.where(np.array(wall_array[mode]) % 2 == 1)
        collision_count_array_prac[user_index][mode_index] = len(prac_wall_indices[0])
        collision_count_array_test[user_index][mode_index] = len(test_wall_indices[0])
        print(f"in mode {mode}, practice collison = {len(prac_wall_indices[0])}, test collison = {len(test_wall_indices[0])}")
        actuator_array[mode] = np.array(actuator_array[mode])[test_wall_indices]
        duration_array[mode] = np.array(duration_array[mode])[test_wall_indices]
        wall_array[mode] = np.array(wall_array[mode])[test_wall_indices]
        shape_array[mode] = np.array(shape_array[mode])[test_wall_indices]
        
    
    # average collision duration time
    for mode_index in range(len(mode_list)):
        mode = mode_list[mode_index]
        collision_duration_array_test[user_index][mode_index] = np.mean(duration_array[mode])
        print(mode, np.mean(duration_array[mode]))
    
    
    # count collisions of each actuator
    unique_actuators, actuator_counts = np.unique(actuator_array["HAPTIC"], return_counts=True)
    for actuator, count in zip(unique_actuators, actuator_counts):
        actuator_collision_count_array[user_index][actuator_ids.index(actuator)] += count

loading OhShape/user3/
load 09-07-2023-02-04-42.json, mode = PRACTICE, collision count = 290, after removing duplicate = 210
load 09-07-2023-02-11-05.json, mode = CLEAN, collision count = 2010, after removing duplicate = 1425
load 09-07-2023-02-25-50.json, mode = VISUAL, collision count = 876, after removing duplicate = 603
load 09-07-2023-02-40-51.json, mode = HAPTIC, collision count = 1472, after removing duplicate = 1108
in mode PRACTICE, practice collison = 146, test collison = 64
in mode CLEAN, practice collison = 690, test collison = 735
in mode VISUAL, practice collison = 295, test collison = 308
in mode HAPTIC, practice collison = 667, test collison = 441
PRACTICE 0.61937415599823
CLEAN 0.6379460548867985
VISUAL 0.7276297730284852
HAPTIC 0.5636976938399058
loading OhShape/user4/
load 09-07-2023-04-50-04.json, mode = PRACTICE, collision count = 926, after removing duplicate = 613
load 09-07-2023-04-56-56.json, mode = VISUAL, collision count = 1678, after removing duplicate = 122

In [10]:
actuator_collision_sum = np.sum(actuator_collision_count_array, axis = 0)
actuator_collision_sum

array([  0,   0,   0,   0,   0,   7,   3,   0,   0,   0,   1,   1,   1,
         2,   5,   0,   0,   0,   0,   0,  29,  20,  21,  32,  65,  51,
        54,  56,  61,  69,  75,  67,  64,  62,  75,  64,  51,  53,  42,
        35,  79, 109, 118, 106,  90, 105,  95,  79,  72,  75,  67,  68,
        63,  65,  60,  72,  53,  64,  65,  57,  32,  30,  30,  33,  60,
        64,  67,  59,  60,  70,  48,  40,  41,  42,  44,  52,  43,  51,
        58,  47,  82, 118, 129, 107,  89,  83,  79,  63,  54,  56,  65,
        63,  66,  72,  72,  62,  41,  48,  53,  40,   0,   0,   0,   5,
         4,   7,   3,   3,  13,   2,   5,   5,  15,   2,   1,   1])

In [11]:
with open('actuator_collision_sum.csv', 'w') as g:
    for i in range(num_actuator):
        g.write(f"{actuator_ids[i]}, {actuator_collision_sum[i]}\n")

In [137]:
res = {
    "actuator_collision_count_array": actuator_collision_count_array.tolist(), 
    "collision_count_array_prac": collision_count_array_prac.tolist(),
    "collision_count_array_test": collision_count_array_test.tolist(),
    "collision_duration_array_test": collision_duration_array_test.tolist()
}

with open("results.json", "w") as g:
    json.dump(res, g, indent = 2)

In [138]:
# collision in practice
collision_count_array_prac = collision_count_array_prac / 24.0
collision_count_array_prac[:, 0] = collision_count_array_prac[:, 0] * 3
print(collision_count_array_prac)
print(np.mean(collision_count_array_prac, axis=0))
print(np.std(collision_count_array_prac, axis=0))

[[18.25       28.75       12.29166667 27.79166667]
 [39.375      42.5        21.91666667 44.79166667]
 [16.625      31.83333333 14.08333333 34.25      ]
 [31.75       41.33333333 28.70833333 27.58333333]
 [28.5        30.04166667 21.58333333 35.83333333]
 [28.875      20.20833333 12.625      21.5       ]
 [29.5        34.08333333 18.70833333 16.91666667]
 [27.         24.875      11.5        19.58333333]
 [31.375      30.04166667 19.41666667 32.16666667]]
[27.91666667 31.51851852 17.87037037 28.93518519]
[6.52771868 6.73039881 5.41881882 8.35345051]


In [139]:
np.savetxt("collision_count_array_prac.csv", collision_count_array_prac, delimiter=",", fmt='%.2f')

In [140]:
# collision in test
collision_count_array_test = collision_count_array_test / 24.0
collision_count_array_test[:, 0] = collision_count_array_test[:, 0] * 3
print(collision_count_array_test)
print(np.mean(collision_count_array_test, axis=0))
print(np.std(collision_count_array_test, axis=0))

[[ 8.         30.625      12.83333333 18.375     ]
 [37.25       44.95833333 29.16666667 42.125     ]
 [15.125      34.45833333 16.04166667 29.29166667]
 [25.75       37.875      29.25       37.41666667]
 [37.375      24.70833333 22.41666667 25.54166667]
 [15.25       18.41666667 10.08333333  9.66666667]
 [13.375      31.375      15.95833333 11.45833333]
 [13.         24.66666667  7.375      15.20833333]
 [23.875      36.04166667 27.25       23.70833333]]
[21.         31.45833333 18.93055556 23.64351852]
[10.12011199  7.57118991  7.87961603 10.59095146]


In [141]:
np.savetxt("collision_count_array_test.csv", collision_count_array_test, delimiter=",", fmt='%.2f')

In [142]:
# collision duration
print(collision_duration_array_test)
print(np.mean(collision_duration_array_test, axis=0))
print(np.std(collision_duration_array_test, axis=0))

[[0.61937416 0.63794607 0.7276298  0.5636977 ]
 [0.77174497 0.81099135 0.6704712  0.7621566 ]
 [0.7217351  0.9390206  0.5277401  0.960569  ]
 [0.7211658  0.9527073  0.8767526  0.9145318 ]
 [0.90990233 0.78131473 0.5925098  0.7909621 ]
 [0.57573885 0.49522617 0.43471122 0.45594913]
 [0.56644934 0.7673444  0.7483559  0.73854584]
 [0.6003876  0.9215192  0.75468934 0.7353967 ]
 [0.68308884 0.794173   0.6912546  0.72676617]]
[0.6855096  0.78891593 0.6693461  0.7387306 ]
[0.10446838 0.1402483  0.12535785 0.1468754 ]


In [143]:
np.savetxt("collision_duration_array_test.csv", collision_duration_array_test, delimiter=",", fmt='%.2f')

In [84]:
    mode_list = ["PRACTICE", "CLEAN", "VISUAL", "HAPTIC"]
    actuator_array = {"PRACTICE":[], "CLEAN":[], "VISUAL":[], "HAPTIC":[]}
    duration_array = {"PRACTICE":[], "CLEAN":[], "VISUAL":[], "HAPTIC":[]}
    wall_array = {"PRACTICE":[], "CLEAN":[], "VISUAL":[], "HAPTIC":[]}
    shape_array = {"PRACTICE":[], "CLEAN":[], "VISUAL":[], "HAPTIC":[]}

In [86]:

json_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) and f.endswith('.json')]

for json_file in json_files:
    with open(os.path.join(folder_path, json_file), 'r') as file:
        data = json.load(file)
    mode = data["mode"]
    shape_indexs = data["shapeIndexs"]
    collision_count = data["numberOfCollusions"]
    collision_data = remove_duplicate_entry(data['collusionData']) # remove duplicate entry
    print(f"load {json_file}, mode = {mode}, collision count = {collision_count}, after removing duplicate = {len(collision_data)}")
    for i in range(len(collision_data)):
        actuator_array[mode].append(collision_data[i]['actutorId'])
        duration_array[mode].append(collision_data[i]['duration'])
        wall_index = int(collision_data[i]['other'].replace('shape', ''))
        wall_array[mode].append(wall_index)
        shape_array[mode].append(shape_indexs[wall_index])

load 09-07-2023-02-04-42.json, mode = PRACTICE, collision count = 290, after removing duplicate = 210
load 09-07-2023-02-11-05.json, mode = CLEAN, collision count = 2010, after removing duplicate = 1425
load 09-07-2023-02-25-50.json, mode = VISUAL, collision count = 876, after removing duplicate = 603
load 09-07-2023-02-40-51.json, mode = HAPTIC, collision count = 1472, after removing duplicate = 1108


In [87]:
# compare the length of practice and test data, showing the learning effect
# only keep the test wall data

for mode in mode_list:
    prac_wall_indices = np.where(np.array(wall_array[mode]) % 2 == 0)
    test_wall_indices = np.where(np.array(wall_array[mode]) % 2 == 1)
    print(f"in mode {mode}, practice collison = {len(prac_wall_indices[0])}, test collison = {len(test_wall_indices[0])}")
    actuator_array[mode] = np.array(actuator_array[mode])[test_wall_indices]
    duration_array[mode] = np.array(duration_array[mode])[test_wall_indices]
    wall_array[mode] = np.array(wall_array[mode])[test_wall_indices]
    shape_array[mode] = np.array(shape_array[mode])[test_wall_indices]

in mode PRACTICE, practice collison = 146, test collison = 64
in mode CLEAN, practice collison = 690, test collison = 735
in mode VISUAL, practice collison = 295, test collison = 308
in mode HAPTIC, practice collison = 667, test collison = 441


In [88]:
# average collision duration time

for mode in mode_list:
    print(mode, np.mean(duration_array[mode]))

PRACTICE 0.61937415599823
CLEAN 0.6379460548867985
VISUAL 0.7276297730284852
HAPTIC 0.5636976938399058


In [94]:
# count collisions of each actuator

unique_actuators, actuator_counts = np.unique(actuator_array["HAPTIC"], return_counts=True)
print(unique_actuators)
print(actuator_counts)

for actuator, count in zip(unique_actuators, actuator_counts):
    actuator_collision_count_array[actuator_ids.index(actuator)] += count

print(actuator_collision_count_array)

[  6  61  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  91
  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109
 110 123 124 125 126 127 128 129 130 131 132 133 134 136 139 140 151 152
 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170
 215]
[ 3  2  2  7  1  6  6  9 12  7  6  4  4  4  6  1  1 11 17 20 23  8  8  9
  9  6  8  7  7  3  3  3  5  4  5  8  8  1  2 15  8  7  4  5  6  3  2  1
  1  4  4  6 12 18 21 10  3  5  5  4  3  4  4  4  5  4  5  7  1  1  1  1
  1]
[ 0  0  0  0  0  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  2
  7  1  6  6  9 12  7  6  4  4  4  6  1  1  0  0 11 17 20 23  8  8  9  9
  6  8  7  7  3  3  3  5  4  5  8  8  0  0  1  2 15  8  7  4  5  6  3  2
  1  1  0  4  0  0  4  6 12 18 21 10  3  5  5  4  3  4  4  4  5  4  5  7
  1  1  1  1  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0]
